In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import dgl
import torch

from collections import defaultdict
from sklearn.decomposition import PCA
import joblib
from dgl.data.utils import save_graphs, load_graphs

In [ ]:
data_postEmb=np.load('/content/drive/MyDrive/BlogAuthorClass/postEmb_384dim_Mono(6).npy', allow_pickle=True)
pca=joblib.load('/content/drive/MyDrive/BlogAuthorClass/PCA_Mono384.pkl')
text=pd.read_csv("/content/drive/MyDrive/BlogAuthorClass/blogtext_2sent.csv")

In [ ]:
window_size=2
device = torch.device('cuda')
post_graph_list=[]

def convert_bool2int(inp):
    return node_bool2int[inp]

for postEmb in data_postEmb:
    node_dict=defaultdict(list)
    src_dict=defaultdict(list)
    dst_dict=defaultdict(list)
    sliding_window=[]
    node_attribute=[]
    
    post_sentwise_clu = pca.transform(postEmb)<0    # dividing latent space for clustering
    for sent_clu, sent_emb in zip(post_sentwise_clu, postEmb):
        sent_clu=tuple(sent_clu)
        node_dict[sent_clu].append(sent_emb)        # node features

        temp_ws=len(sliding_window)
        for i in range(temp_ws):
            src_dict[str(i+1)+"hop"].append(sliding_window[temp_ws-1-i])
            dst_dict[str(i+1)+"hop"].append(sent_clu)
            
            
        sliding_window.append(sent_clu)
        if len(sliding_window) > window_size:
            sliding_window.pop(0)

            
    sent_nodes = node_dict.keys()
    node_bool2int = defaultdict()
    for nodeBool,nodeInt in zip(sent_nodes, range(len(sent_nodes))):
        node_bool2int[nodeBool]=nodeInt
        node_attribute.append(sum(node_dict[nodeBool])/len(node_dict[nodeBool]))
        
    
    for i in range(window_size):
        src_dict[str(i+1)+"hop"]=torch.tensor(list(map(convert_bool2int,src_dict[str(i+1)+"hop"])))
        dst_dict[str(i+1)+"hop"]=torch.tensor(list(map(convert_bool2int,dst_dict[str(i+1)+"hop"])))
    
    g=defaultdict()
    for i in range(window_size):
        g[("f", str(i+1)+"hop", "f")] = (src_dict[str(i+1)+"hop"],dst_dict[str(i+1)+"hop"])
    
    try :
        g=dgl.heterograph(g).to_simple(return_counts="overlap").to(device)
        g.ndata['f'] = torch.tensor(node_attribute, device='cuda')
    except :   # for cases which has fewer or same node numbers then "window_size"
        g.popitem()
        g=dgl.heterograph(g).to_simple(return_counts="overlap").to(device)
        g.ndata['f'] = torch.tensor(node_attribute, device='cuda')
    
    post_graph_list.append(g)

In [ ]:
save_graphs("/content/drive/MyDrive/BlogAuthorClass/post_graph_list(6).bin", post_graph_list)

In [ ]:
post_graph_list = load_graphs("/content/drive/MyDrive/BlogAuthorClass/post_graph_list(total).bin")[0]

In [ ]:
nx.draw_kamada_kawai(dgl.to_homogeneous(post_graph_list[43310]).cpu().to_networkx(), node_size=77)

In [ ]:
nx.draw_kamada_kawai(post_graph_list[43310][('f','1hop','f')].cpu().to_networkx(), node_size=77)

In [ ]:
save_graphs("/content/drive/MyDrive/BlogAuthorClass/post_graph_list(total).bin", post_graph_list)

In [ ]:
post_graph_list = load_graphs("/content/drive/MyDrive/BlogAuthorClass/post_graph_list(total).bin")[0]